# Hartree-Fock Theory

**This assignment is due on Friday, January 24 at 5PM by email.**

This partial walkthrough assumes you have already familiar with the math of Hatree-Fock theory.

The Hartree-Fock method is typically used to solve the time-independent Schrodinger equation for a multi-electron atom or molecule as described in the Born-Oppenheimer approximation. Since there are no known solutions for many-electron systems (hydrogenic atoms and the diatomic hydrogen cation being notable one-electron exceptions), the problem is solved numerically.

## Theory
### The Closed-Shell Hartree-Fock Energy
The electronic energy in the MO basis
\begin{align}
E_{elec}^{clscf}  = &  E_{one} \ +\  E_{two}   \\
 = &  2 \ \sum_i^n \  h_{ii} \ +\  \sum_{ij}^n \  \biggl\{ 2(ii|jj) \ -\  (ij|ij) \biggr\} \\
 = &  2 \ \sum_i^n \ h_{ii} \ + \ \sum_{ij}^n \  \biggl\{ 2J_{ij} \ -\  K_{ij} \biggr\} \\
 = &  \sum_i^n \  \biggl( h_{ii} \ +\  F_{ii} \biggr) \\
\end{align}

The density matrix
\begin{equation}
D_{\mu\nu} = \sum_i^{\mathrm{d.o.}} C_\mu^i C_\nu^i
\end{equation}

The one-electron energy
\begin{align}
E_{one} = &   2 \ \sum_i^n \  h_{ii} \\
 = &  2 \ \sum_i^n \  \left\{ \sum_{ \mu \nu }^{AO} \  C_{\mu}^i C_{\nu}^i\ h_{ \mu \nu } \right\} \\
 = &  2 \ \sum_{ \mu \nu }^{AO} \  \left\{ \sum_i^n \  C_{\mu}^i C_{\nu}^i  \right\}  h_{ \mu \nu } \\
 = &  2 \ \sum_{ \mu \nu }^{AO} \  D_{ \mu \nu }   h_{ \mu \nu }
\end{align}

The two-electron energy
\begin{align}
E_{two} = &   \sum_{ij}^n \  \biggl\{ 2 (ii|jj) \ -\  (ij|ij) \biggr\} \\
 = &  \sum_{ij}^n \  \Biggl[ \sum_{ \mu \nu \rho \sigma }^{AO} \  \biggl\{ 2 C_{\mu}^i C_{\nu}^i C_{\rho}^j 
C_{\sigma}^j\ ( \mu \nu | \rho \sigma )
\ -\  C_{\mu}^i C_{\nu}^j C_{\rho}^i C_{\sigma}^j\ ( \mu \nu | \rho \sigma ) \biggr\}  \Biggr]
 \nonumber  \\
   \\
 = &  \sum_{ \mu \nu \rho \sigma }^{AO} \  \Biggl[ \biggl\{ \sum_i^n \  C_{\mu}^i C_{\nu}^i \biggr\} 
\biggl\{ \sum_j^n \  C_{\rho}^j C_{\sigma}^j \biggr\}
   \biggl\{  2 ( \mu \nu | \rho \sigma ) \ -\  ( \mu \rho | \nu \sigma ) \biggr\}
\Biggr] \\
 = & \sum_{ \mu \nu \rho \sigma }^{AO} \  D_{ \mu \nu } D_{ \rho \sigma }
    \biggl\{ 2 ( \mu \nu | \rho \sigma )
\ -\  ( \mu \rho | \nu \sigma ) \biggr\}
\end{align}

The electronic energy in the AO basis
\begin{align}
E_{elec}^{clscf} = & \ 2 \ \sum_{ \mu \nu }^{AO} \  D_{ \mu \nu } h_{ \mu \nu }
\ +\  \sum_{ \mu \nu \rho \sigma }^{AO} \  D_{ \mu \nu } D_{ \rho \sigma }  
\biggl\{ 2 ( \mu \nu | \rho \sigma ) \ -\  ( \mu \rho | \nu \sigma ) \biggr\} \nonumber \\
   \\
 = &  \sum_{ \mu \nu }^{AO} \  D_{ \mu \nu }  \Biggl[   2 h_{ \mu \nu }
\ +\  \sum_{ \rho \sigma }^{AO} \  D_{ \rho \sigma } 
  \biggl\{ 2 ( \mu \nu | \rho \sigma ) \ -\  ( \mu \rho | \nu \sigma ) \biggr\} \Biggr] \\
 = &  \sum_{ \mu \nu }^{AO} \  D_{ \mu \nu }   \biggl(   h_{ \mu \nu } \ +\  F_{ \mu \nu } \biggr)
\end{align}
\begin{align}
F_{ \mu \nu } = \ h_{ \mu \nu }
\ +\  \sum_{ \rho \sigma }^{AO} \  D_{ \rho \sigma } 
  \biggl\{ 2 ( \mu \nu | \rho \sigma ) \ -\  ( \mu \rho | \nu \sigma ) \biggr\} \Biggr]
\end{align}

## Load <span style="font-variant: small-caps">Psi4</span> and NumPy
You will use Psi4 to compute integrals and NumPy to perform the linear algebra.

Input options will be provided by using a separate Python file `input.py`. Sample contents:

    Settings = dict()

    Settings["basis"] = "STO-3G"
    Settings["molecule"] = """
      0 1
      O
      H 1 R
      H 1 R 2 A
      R = 1.0
      A = 104.5
      symmetry c1
    """
    Settings["nalpha"] = 5
    Settings["nbeta"] = 5
    Settings["scf_max_iter"] = 50

In [2]:
import psi4
import numpy as np
from input import Settings

print(Settings)

np.set_printoptions(suppress=True, linewidth=120)

{'basis': 'STO-3G', 'molecule': '\n  0 1\n  O\n  H 1 R\n  H 1 R 2 A\n  R = 1.0\n  A = 104.5\n  symmetry c1\n', 'nalpha': 5, 'nbeta': 5, 'scf_max_iter': 50}


## Setup your initial conditions
- Construct a new molecule in C1 symmetry
- Load in the basis set
- Create a `MintsHelper` object to will compute the integrals. Refer to [this](http://www.psicode.org/psi4manual/master/api/psi4.core.MintsHelper.html) for grossly incomplete documentation.

In [3]:
molecule = psi4.geometry(Settings['molecule'])
molecule.update_geometry()

ndocc = Settings['nalpha']

scf_max_iter = Settings['scf_max_iter']

print(molecule.nuclear_repulsion_energy())

basis = psi4.core.BasisSet.build(molecule, 'BASIS', Settings['basis'])

mints = psi4.core.MintsHelper(basis)

8.801465564567374


## Compute the needed integrals

In [4]:
# Overlap
S = mints.ao_overlap().np

# Kinetic
T = mints.ao_kinetic().np

# Potential
V = mints.ao_potential().np

# Two-electron repulsion
I = mints.ao_eri().np

print(I.shape)

(7, 7, 7, 7)


## Form the one-electron Hamiltonian (H)
\begin{equation}
H_{\mu\nu} = T_{\mu\nu} + V_{\mu\nu}
\end{equation}

In [5]:
H = T + V

print(H)

[[-32.67360938  -7.60160704  -0.01741536   0.           0.          -1.57818038  -1.57818038]
 [ -7.60160704  -9.2914758   -0.20781331   0.          -0.          -3.47334591  -3.47334591]
 [ -0.01741536  -0.20781331  -7.50695983   0.           0.          -1.55452724  -1.55452724]
 [  0.           0.           0.          -7.42214005   0.           0.           0.        ]
 [  0.          -0.           0.           0.          -7.56362101   1.92474988  -1.92474988]
 [ -1.57818038  -3.47334591  -1.55452724   0.           1.92474988  -4.90357867  -1.42265626]
 [ -1.57818038  -3.47334591  -1.55452724   0.          -1.92474988  -1.42265626  -4.90357867]]


## Construct the orthogonalizer $S^{-1/2}$
The steps to complete this are:
- Diagonalize the S matrix
\begin{align}
& S L_S = L_S \Lambda_S \\
& L_S \tilde L_S = L_S L_S^{-1} = 1
\end{align}
- Form the $S^{-1/2}$ matrix
\begin{equation}
S^{-1/2} = L_S \Lambda_S^{-1/2} \tilde L_S
\end{equation}

Or use the <span style="font-variant: small-caps">Psi4</span> function  ```power``` which does all of this for you.

In [6]:
A = mints.ao_overlap()
A.power(-0.5, 1.e-16)
A = A.np
print(A)

[[ 1.02403592 -0.14128825 -0.00938732  0.          0.          0.02117023  0.02117023]
 [-0.14128825  1.21591378  0.09833518  0.         -0.         -0.27163956 -0.27163956]
 [-0.00938732  0.09833518  1.04984867  0.         -0.         -0.1381764  -0.1381764 ]
 [ 0.          0.          0.          1.         -0.         -0.         -0.        ]
 [ 0.         -0.         -0.         -0.          1.10025748  0.2134062  -0.2134062 ]
 [ 0.02117023 -0.27163956 -0.1381764  -0.          0.2134062   1.1823526  -0.08078394]
 [ 0.02117023 -0.27163956 -0.1381764  -0.         -0.2134062  -0.08078394  1.1823526 ]]


## Construct the initial density matrix
In this sample, we will be using a core Hamiltonian guess.  Perform the following steps:

- Form an initial (transformed) $F_0^\tau$ matrix using the H matrix.
\begin{equation}
F_0^\tau = \tilde{S}^{-1/2} H S^{-1/2}
\end{equation}
- Diagonalize the $F_0^\tau$ matrix using a standard eigenvalue routine.
\begin{equation}
F_0^\tau C_0^\tau = C_0^\tau \epsilon
\end{equation}
- Form the SCF eigenvector matrix
\begin{equation}
C = S^{-1/2} C_0^\tau
\end{equation}
- Form the first density matrix (D) using the doubly occupied orbitals from the eigenvector matrix
\begin{equation}
D_{\mu\nu} = \sum_{m}^{\mathrm{d.o.}} C_\mu^m C_\nu^m
\end{equation}

In [7]:
Ft = A.dot(H).dot(A)
_, C = np.linalg.eigh(Ft)
C = A.dot(C)
Cocc = C[:, :ndocc]
D = np.einsum('pi,qi->pq', Cocc, Cocc)

print(D)

[[ 1.06661727 -0.29755998 -0.02536219  0.          0.          0.04007352  0.04007352]
 [-0.29755998  1.32589965  0.15348429  0.         -0.         -0.17684186 -0.17684186]
 [-0.02536219  0.15348429  1.08573619 -0.         -0.         -0.11682694 -0.11682694]
 [ 0.          0.         -0.          1.         -0.         -0.          0.        ]
 [ 0.         -0.         -0.         -0.          1.17091368  0.1776736  -0.1776736 ]
 [ 0.04007352 -0.17684186 -0.11682694 -0.          0.1776736   0.05924138  0.00532125]
 [ 0.04007352 -0.17684186 -0.11682694  0.         -0.1776736   0.00532125  0.05924138]]


## SCF Iterations
Within the SCF iteration we must perform the following steps.
- Construct a new Fock matrix (F) include the two-electron repulsion integrals
\begin{equation}
F_{\mu\nu} = H_{\mu\nu} + \sum_{\rho\sigma}^{AO} D_{\rho\sigma} \left[ 2 (\mu\nu \mid \rho\sigma) - (\mu\rho \mid \nu\sigma) \right]
\end{equation}
- Calculate this iteration's electronic and total energies
\begin{equation}
E_{elec} = \sum_{\mu\nu}^{AO} D_{\mu\nu} \left( H_{\mu\nu} + F_{\mu\nu} \right)
\end{equation}
\begin{equation}
E_{total} = E_{elec} + E_{nuc}
\end{equation}
- Transform the Fock matrix
\begin{equation}
F^\tau = \tilde{S}^{-1/2} F S^{-1/2}
\end{equation}
- Diagonalize the Fock matrix
\begin{equation}
F^\tau C^\tau = C^\tau \epsilon
\end{equation}
- Construct the new SCF eigenvector matrix
\begin{equation}
C = S^{-1/2} C^\tau
\end{equation}
- Form the new density matrix
\begin{equation}
D_{\mu\nu} = \sum_{m}^{d.o.} C_\mu^m C_\nu^m
\end{equation}
- Test convergency of the density matrix and energy
\begin{equation}
rms = \left[ \sum_{\mu\nu}^{AO} \left( D_{\mu\nu}^n - D_{\mu\nu}^{n-1} \right)^2 \right]^{1/2} < \delta_1
\end{equation}
\begin{equation}
\Delta E = E_{SCF}^{n} - E_{SCF}^{n-1} < \delta_2
\end{equation}

In [8]:
J = np.einsum('pqrs,rs->pq', I, D)
K = np.einsum('prqs,rs->pq', I, D)
F = H + J * 2 - K
print(F)

[[-18.85371222  -4.88435988  -0.01396825  -0.          -0.          -1.02412265  -1.02412265]
 [ -4.88435988  -1.86077282  -0.21349587  -0.           0.          -0.71030031  -0.71030031]
 [ -0.01396825  -0.21349587   0.1682888    0.           0.          -0.21769452  -0.21769452]
 [ -0.          -0.           0.           0.25498122   0.           0.          -0.        ]
 [ -0.           0.           0.           0.           0.10983997   0.19510018  -0.19510018]
 [ -1.02412265  -0.71030031  -0.21769452   0.           0.19510018  -0.21971495  -0.25918679]
 [ -1.02412265  -0.71030031  -0.21769452  -0.          -0.19510018  -0.25918679  -0.21971495]]


With my code the ouput is the following:

    RHF iteration   1: energy   -73.25301168397348  dE -7.32530E+01
    RHF iteration   2: energy   -74.93149651086453  dE -1.67848E+00
    RHF iteration   3: energy   -74.96316771179171  dE -3.16712E-02
    RHF iteration   4: energy   -74.96447365205802  dE -1.30594E-03
    RHF iteration   5: energy   -74.96462787384583  dE -1.54222E-04
    RHF iteration   6: energy   -74.96465575231846  dE -2.78785E-05
    RHF iteration   7: energy   -74.96466118823379  dE -5.43592E-06
    RHF iteration   8: energy   -74.96466226898241  dE -1.08075E-06
    RHF iteration   9: energy   -74.96466248501822  dE -2.16036E-07
    RHF iteration  10: energy   -74.96466252827092  dE -4.32527E-08
    RHF iteration  11: energy   -74.96466253693451  dE -8.66359E-09
    RHF iteration  12: energy   -74.96466253867011  dE -1.73560E-09
    RHF iteration  13: energy   -74.96466253901779  dE -3.47683E-10
    RHF iteration  14: energy   -74.96466253908747  dE -6.96758E-11

## Test against <span style="font-variant: small-caps">Psi4</span>

In [8]:
# The energy that you obtain must match the energy that Psi4 produces (at least
# to the extent that you converged the energy). If you do not match then check 
# your code for bugs!
psi4.set_options({'basis': 'sto-3g',
                  'scf_type': 'pk',
                  'e_convergence': 1e-10,
                 'reference': 'uhf'})
psi4.energy('scf')

-74.96466253910498